In [1]:
from IPython.display import display, Markdown

notebook_sections = [
    "## 🩺 1. Oura API Data Ingestion\n"
    "**What this does:**\n"
    "- Connects to the [Oura Ring API v2](https://cloud.ouraring.com/v2/docs)\n"
    "- Pulls your daily **readiness**, **sleep**, and **activity** scores\n"
    "- Stores as a single clean `DataFrame`\n"
    "- Adds basic GPT-style `insight` logic (or placeholder)\n\n"
    "**You’ll need:**\n"
    "- Oura Personal Access Token (PAT) stored as a string.\n"
    "- Example endpoint: `https://api.ouraring.com/v2/usercollection/daily_readiness`\n",

    "## 🧠  2. LLM GPT\n",

    "## 📤 3. Google Sheets Sync\n"
    "**What this does:**\n"
    "- Uses `gspread` and `google-auth` to write your daily metrics to Google Sheets\n"
    "- Creates a sheet titled `'Health_Log'` if it doesn’t exist\n"
    "- Appends new rows only if the date isn't already present\n",

    "## 🧠 4. Chroma Vector Memory Setup\n"
    "**What this does:**\n"
    "- Converts each row of your health data into an embedded memory chunk\n"
    "- Stores using `Chroma` with `OpenAIEmbeddings`\n"
    "- Persisted to `./chroma_health_memory` so it's reusable across sessions\n",

    "## 🤖 5. LangChain Agent Setup\n"
    "**What this does:**\n"
    "- Creates a GPT-4-powered agent\n"
    "- Wraps your `Chroma` retriever as a `Tool`\n"
    "- Uses `create_openai_functions_agent()` and `AgentExecutor`\n"
    "- Handles natural language queries like “When was my best recovery?”\n",

    "## 🌐 6. Streamlit UI (in Colab via `pyngrok`)\n"
    "**What this does:**\n"
    "- Provides a simple web UI using `Streamlit`\n"
    "- Lets you type in natural questions\n"
    "- Answers appear after being processed by GPT + your memory\n"
    "- UI launched via `ngrok` and previewed in Colab\n"
]

# Display sections as Markdown overview
for section in notebook_sections:
    display(Markdown(section))



## 🩺 1. Oura API Data Ingestion
**What this does:**
- Connects to the [Oura Ring API v2](https://cloud.ouraring.com/v2/docs)
- Pulls your daily **readiness**, **sleep**, and **activity** scores
- Stores as a single clean `DataFrame`
- Adds basic GPT-style `insight` logic (or placeholder)

**You’ll need:**
- Oura Personal Access Token (PAT) stored as a string.
- Example endpoint: `https://api.ouraring.com/v2/usercollection/daily_readiness`


## 🧠  2. LLM GPT


## 📤 3. Google Sheets Sync
**What this does:**
- Uses `gspread` and `google-auth` to write your daily metrics to Google Sheets
- Creates a sheet titled `'Health_Log'` if it doesn’t exist
- Appends new rows only if the date isn't already present


## 🧠 4. Chroma Vector Memory Setup
**What this does:**
- Converts each row of your health data into an embedded memory chunk
- Stores using `Chroma` with `OpenAIEmbeddings`
- Persisted to `./chroma_health_memory` so it's reusable across sessions


## 🤖 5. LangChain Agent Setup
**What this does:**
- Creates a GPT-4-powered agent
- Wraps your `Chroma` retriever as a `Tool`
- Uses `create_openai_functions_agent()` and `AgentExecutor`
- Handles natural language queries like “When was my best recovery?”


## 🌐 6. Streamlit UI (in Colab via `pyngrok`)
**What this does:**
- Provides a simple web UI using `Streamlit`
- Lets you type in natural questions
- Answers appear after being processed by GPT + your memory
- UI launched via `ngrok` and previewed in Colab


In [2]:
#All pip
!pip install requests pandas openai
!pip install gspread gspread_dataframe oauth2client
!pip install chromadb langchain-openai langchain-community
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 12.3 MB/s eta 0:

In [ ]:
# ------------------------------------------
# 1. Oura API → DataFrame
# ------------------------------------------
import requests
import pandas as pd
from datetime import date, timedelta

OURA_TOKEN = "zzz" # Replace this with your Oura API token
headers = {"Authorization": f"Bearer {OURA_TOKEN}"}

# 📅 Pull last 7 days
end_date = date.today()
start_date = end_date - timedelta(days=7)

# 🔁 API fetch function
def fetch_oura_data(endpoint):
    url = f"https://api.ouraring.com/v2/usercollection/{endpoint}"
    headers = {"Authorization": f"Bearer {OURA_TOKEN}"}
    params = {
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat()
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()['data']

  # 🚀 Download all three endpoints
readiness = pd.DataFrame(fetch_oura_data("daily_readiness"))
sleep     = pd.DataFrame(fetch_oura_data("daily_sleep"))
activity  = pd.DataFrame(fetch_oura_data("daily_activity"))

# Convert datetime columns to date
readiness['date'] = pd.to_datetime(readiness['timestamp']).dt.date
sleep['date'] = pd.to_datetime(sleep['timestamp']).dt.date
activity['date'] = pd.to_datetime(activity['timestamp']).dt.date

# Select and rename key fields
df = (
    readiness[['date', 'score']]
    .rename(columns={'score': 'readiness_score'})
    .merge(
        sleep[['date', 'score']].rename(
            columns={'score': 'sleep_score'}
        ),
        on='date',
        how='outer'
    )
    .merge(
        activity[['date', 'score', 'steps', 'active_calories']].rename(
            columns={'score': 'activity_score'}
        ),
        on='date',
        how='outer'
    )
)

# Convert total sleep duration from seconds to hours
# Since total_sleep_duration is not available, this step will be skipped or adjusted if a similar column exists in activity data

# Final clean-up
df = df[['date', 'readiness_score', 'sleep_score',
         'activity_score', 'steps', 'active_calories']].sort_values('date')

df.reset_index(drop=True, inplace=True)
df.head(10)

,date,readiness_score,sleep_score,activity_score,steps,active_calories
0,2025-09-12,80,91,84,5189,261
1,2025-09-13,72,67,86,8090,374
2,2025-09-14,82,77,88,9077,414
3,2025-09-15,81,83,94,10740,395
4,2025-09-16,80,85,88,5682,208
5,2025-09-17,83,83,94,7618,371
6,2025-09-18,87,84,93,7665,495


In [ ]:
# ------------------------------------------
# 2. GPT insights → DataFrame
# ------------------------------------------
from openai import OpenAI

client = OpenAI(api_key="zzz")  # Replace with your real key

def gpt_health_summary(row):
    prompt = f"""
You are a helpful health coach. Based on this user's daily biometric and sleep/activity data, write a 1–2 sentence personalized wellness insight.

Data:
- Date: {row['date']}
- Readiness Score: {row['readiness_score']}
- Sleep Score: {row['sleep_score']}
- Activity Score: {row['activity_score']}
- Steps: {row['steps']}
- Active Calories: {row['active_calories']}

Keep it supportive and encouraging, and speak like a real wellness coach.
"""
    response = client.chat.completions.create(
        model="gpt-4",  # or "gpt-3.5-turbo"
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content
# Apply the gpt_health_summary function to each row of the DataFrame
df['gpt_insight'] = df.apply(gpt_health_summary, axis=1)

In [5]:
# ------------------------------------------
# 3. Google Sheets Sync
# ------------------------------------------
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# Authenticate and create the client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet_title = "Oura Ring Data and Insights"
spreadsheet = gc.create(spreadsheet_title)
print(f"📄 Spreadsheet created: {spreadsheet_title}")
print(f"🔗 Link: {spreadsheet.url}")

worksheet = spreadsheet.sheet1

# Select the desired columns from df and assign to memory_df
memory_df = df[['date', 'readiness_score', 'sleep_score', 'activity_score', 'steps', 'active_calories','gpt_insight']].copy()

# Write DataFrame into the sheet
set_with_dataframe(worksheet, memory_df)
print("✅ Agent memory pushed to Google Sheets!")

📄 Spreadsheet created: Oura Ring Data and Insights
🔗 Link: https://docs.google.com/spreadsheets/d/1O2kVFj1tM9Je8OzoQza6cgEbwPm5wkM06mqJhFcVaNs
✅ Agent memory pushed to Google Sheets!


In [ ]:
# ------------------------------------------
# 4. Chroma Vector Memory Setup
# ------------------------------------------
import os
os.environ["OPENAI_API_KEY"] = "zzz"  # your actual key here
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

texts = [
f"Date: {row['date']}, Readiness: {row['readiness_score']}, Sleep: {row['sleep_score']}, Activity: {row['activity_score']}, Steps: {row['steps']}, Calories: {row['active_calories']}, Insight: {row['gpt_insight']}"
for _, row in df.iterrows()
]
metas = [{"date": row["date"]} for _, row in df.iterrows()]

vectorstore = Chroma.from_texts(
texts=texts,
embedding=OpenAIEmbeddings(),
metadatas=[{"date": str(d)} for d in df["date"]],
persist_directory="./chroma_health_memory"
)
retriever = vectorstore.as_retriever()

/tmp/ipython-input-450112033.py:18: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings(),


In [9]:
# ------------------------------------------
# 5. LangChain Agent Setup
# ------------------------------------------
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_openai_functions_agent, AgentExecutor


# Step 1: Wrap your retriever
memory_tool = create_retriever_tool(
    retriever,
    name="HealthMemorySearch",
    description="Search historical recovery, sleep, readiness, and GPT insight data"
)

# Step 2: Define LLM
llm = ChatOpenAI(model="gpt-4", temperature=0.3)

# Step 3: Define Agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful health memory agent."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_openai_functions_agent(
    llm=llm,
    tools=[memory_tool],  # your retriever-based tool
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=[memory_tool], verbose=True)


/tmp/ipython-input-3649821757.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0.3)


In [10]:
query = "What was my best recovery day this week?"
response = agent_executor.invoke({"input": query})
print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `HealthMemorySearch` with `{'query': 'best recovery day this week'}`


Date: 2025-09-14, Readiness: 82, Sleep: 77, Activity: 88, Steps: 9077, Calories: 414, Insight: Great job on maintaining a high activity score and steps yesterday! However, your sleep score is slightly lower, so try to prioritize rest tonight to improve overall readiness and performance.

Date: 2025-09-17, Readiness: 83, Sleep: 83, Activity: 94, Steps: 7618, Calories: 371, Insight: Great job on maintaining a balanced wellness routine on September 17th, 2025! Your readiness, sleep, and activity scores are all high, indicating your body is responding well to the healthy habits you've been cultivating - keep it up!

Date: 2025-09-18, Readiness: 87, Sleep: 84, Activity: 93, Steps: 7665, Calories: 495, Insight: Fantastic job on maintaining high scores in readiness, sleep, and activity on Sep 18, 2025! You're striking a great balance, just make sure to keep up with your st

In [11]:
# ------------------------------------------
# 6. Streamlit → UI
# ------------------------------------------
code = '''
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.agents import Tool, create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

st.set_page_config(page_title="🧠 Health Memory Agent")
st.title("🧠 Health Memory Agent")
query = st.text_input("Ask about your recovery, sleep, or steps:")

if query:
    vectorstore = Chroma(persist_directory="./chroma_health_memory", embedding_function=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()

    tool = Tool.from_function(
        func=retriever.invoke,
        name="HealthMemorySearch",
        description="Search historical recovery, sleep, and step logs"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful health memory agent."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])

    agent = create_openai_functions_agent(ChatOpenAI(model="gpt-4"), [tool], prompt)
    executor = AgentExecutor(agent=agent, tools=[tool], verbose=False)
    result = executor.invoke({"input": query})
    st.success(result["output"])
'''

# Save to file
with open("streamlit_app.py", "w") as f:
    f.write(code)

In [ ]:
# Install ngrok and streamlit if not done
from pyngrok import conf, ngrok

conf.get_default().auth_token = "zzz"

# Kill previous tunnels if any
ngrok.kill()

# Run the app in background
get_ipython().system_raw('streamlit run streamlit_app.py &')

# Get public URL
public_url = ngrok.connect(8501)
print(f"🌐 Streamlit app is live at: {public_url}")

🌐 Streamlit app is live at: NgrokTunnel: "https://1eb7b1eba697.ngrok-free.app" -> "http://localhost:8501"
